## Introduction

In this notebook, we'll walk through the process of building a simple chatbot using several powerful tools:

- LangChain: A framework for developing applications powered by language models.
- Amazon Bedrock: A service providing access to pre-trained large language models (LLMs).
- DynamoDB: A fully managed NoSQL database service from AWS, used here to store chat history.
- Streamlit: An open-source app framework used to create interactive web apps with Python.

We'll integrate these components to build a conversational AI that can store and retrieve messages across sessions, ensuring continuity in conversation.
### Requirements

Before we begin, make sure you have the required Python packages installed. Run the following command:

In [1]:
! pip install --upgrade langchain_aws langchain_community langchain_core streamlit --quiet

## Setting Up Bedrock with LangChain

The first step in building our chatbot is to connect to Amazon Bedrock. Bedrock allows us to use pre-trained language models to generate text responses. We'll be using the `ChatBedrockConverse` class from the `langchain_aws` package to interface with the Bedrock service.

### Creating a Bedrock Model Instance

We start by creating an instance of the `ChatBedrockConverse` class, which will allow us to send prompts to the Bedrock service and receive responses generated by the Anthropic Claude model.

In [2]:
from langchain_aws import ChatBedrockConverse

model = ChatBedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    max_tokens=2048,
    temperature=0.0,
    top_p=1,
    stop_sequences=["\n\nHuman"],
    verbose=True
)

### Testing the Bedrock Model

To understand how the model works and ensure it’s correctly set up, we’ll perform some basic tests by invoking the model with simple prompts.

In [3]:
response = model.invoke(input="Hi! I'm Bob")
print(response)

content="It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know if there's anything I can assist you with." response_metadata={'ResponseMetadata': {'RequestId': '79734851-bb3c-420f-8835-b441a0417b90', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 09 Aug 2024 15:06:16 GMT', 'content-type': 'application/json', 'content-length': '387', 'connection': 'keep-alive', 'x-amzn-requestid': '79734851-bb3c-420f-8835-b441a0417b90'}, 'RetryAttempts': 0}, 'stopReason': 'stop_sequence', 'metrics': {'latencyMs': 1118}} id='run-cc12e5a1-a09b-4d80-81e2-dce56531cfe3-0' usage_metadata={'input_tokens': 12, 'output_tokens': 52, 'total_tokens': 64}


In [4]:
response = model.invoke(input="What's my name?")
print(response)

content="I'm afraid I don't actually know your name. As an AI assistant, I don't have personal information about you unless you provide it to me directly." response_metadata={'ResponseMetadata': {'RequestId': 'e588c3fa-b7c2-472d-a0f0-f65411d00241', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 09 Aug 2024 15:06:19 GMT', 'content-type': 'application/json', 'content-length': '331', 'connection': 'keep-alive', 'x-amzn-requestid': 'e588c3fa-b7c2-472d-a0f0-f65411d00241'}, 'RetryAttempts': 0}, 'stopReason': 'stop_sequence', 'metrics': {'latencyMs': 688}} id='run-d65ea2c7-555b-4ac4-94a7-18bc7002b463-0' usage_metadata={'input_tokens': 12, 'output_tokens': 34, 'total_tokens': 46}


### Explanation:

#### Purpose of the Test:
The first test (Hi! I'm Bob) introduces the user to the model. Since this is the first interaction, the model doesn't have any previous context about who "Bob" is.

The second test (What's my name?) asks the model to recall the name from the first input. However, because there’s no conversation context provided to the model, it should fail to recognize the user's name.

#### Expected Outcome:
The model will not recognize the user's name in the second prompt because there is no chat history or context being passed along with the input. This demonstrates the need for a persistent conversation history, which we will address by integrating DynamoDB.

## Setting Up DynamoDB for Chat History

Now that we've established that the model needs context to remember past interactions, we’ll set up a DynamoDB table. This table will store the chat history, allowing our chatbot to maintain a continuous conversation over multiple interactions.

### Creating the DynamoDB Table

We’ll use Python boto3 client to create a table that will store each user session’s chat history. The table will use the `SessionId` as the primary key to ensure that each session’s history is stored separately.

In [5]:
import boto3
dynamodb = boto3.resource("dynamodb")
client = boto3.client('sts')

try:
    # Create the DynamoDB table.
    table = dynamodb.create_table(
        TableName="SessionTable",
        KeySchema=[{"AttributeName": "SessionId", "KeyType": "HASH"}],
        AttributeDefinitions=[{"AttributeName": "SessionId", "AttributeType": "S"}],
        BillingMode="PAY_PER_REQUEST",
    )
    # Wait until the table exists.
    table.meta.client.get_waiter("table_exists").wait(TableName="SessionTable")

    # Print out some data about the table.
    print(table.item_count)
except Exception as e:
    print(e)

0


## Retrieving the IAM User ID

To uniquely identify each session, we’ll use the IAM User ID associated with the AWS account running this application. This ensures that each user's chat history is stored separately and securely.
- The IAM User ID is a unique identifier for the AWS user or role under which the application is running. By using this ID as the session identifier, we ensure that each user or role has a distinct and separate conversation history.
- This approach also enhances security and auditability, as each session is tied to a specific AWS identity


In [6]:
def get_iam_user_id():    
    # Get the caller identity
    identity = client.get_caller_identity()
    
    # Extract and return the User ID
    user_id = identity['UserId']
    return user_id

# Get UserId for sessionId
user_id = get_iam_user_id()

## Integrating LangChain with DynamoDB

With our DynamoDB table ready, we can now integrate it with LangChain. This will enable our chatbot to store and retrieve conversation history, allowing it to maintain context across multiple interactions.

### Setting Up the Chat History and Prompt System

We'll use the `DynamoDBChatMessageHistory` class to interact with DynamoDB and store chat messages. The LangChain prompt system will then use this stored history to provide context to the Bedrock model.

In [7]:
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
import logging

logging.basicConfig(level=logging.CRITICAL)

# Initialize the DynamoDB chat message history
history = DynamoDBChatMessageHistory(table_name="SessionTable", session_id="0")

# Create the chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# Create output parser to simplify the output
output_parser = StrOutputParser()

# Combine the prompt with the Bedrock LLM
chain = prompt | model| output_parser

# Integrate with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=session_id
    ),
    input_messages_key="question",
    history_messages_key="history",
)

# Invoke the chain with a session-specific configuration
config = {"configurable": {"session_id": user_id}}

### Testing the Integrated System

Now, let's test the integration by sending some prompts and verifying that the model can maintain context across different interactions.

In [8]:
response = chain_with_history.invoke({"question": "Hi! I'm Bob"}, config=config)
print(response)

It's nice to meet you, Bob! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you might have. Please let me know if there's anything I can assist you with.


In [9]:
response = chain_with_history.invoke({"question": "What's my name?"}, config=config)
print(response)

Your name is Bob, as you introduced yourself to me earlier.


### Explanation:

#### Purpose of the Test:
This time, when we ask "What's my name?" after introducing ourselves as "Bob", the chatbot should remember the name because the context is stored in DynamoDB and passed along with each prompt.
#### Expected Outcome:
The chatbot should correctly identify the user's name as "Bob" in the second prompt, demonstrating that it now maintains context across interactions.

## Creating a Streamlit Interface

To interact with our chatbot in a user-friendly way, we’ll build a simple web interface using Streamlit. This interface will allow us to input questions and see responses in real-time, with the context being managed by DynamoDB.

### Step 1: Initial Streamlit Setup

We start by creating a basic echo bot to familiarize ourselves with Streamlit. This will later be replaced by our AI-driven chatbot.

In [10]:
%%writefile app.py
import streamlit as st

st.title("Echo Bot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("What is up?"):
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

response = f"Echo: {prompt}"
# Display assistant response in chat message container
with st.chat_message("assistant"):
    st.markdown(response)
# Add assistant response to chat history
st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting app.py


## Running the Application

Finally, you can run your Streamlit application with the following command. This will launch the Streamlit application and allow you to interact with the chatbot through a web interface.

Take your notebook URL from the current web page and format it like the following in another tab in your internet browser:

`https://<unique_notebook_id>.notebook.us-west-2.sagemaker.aws/proxy/absolute/8501`

In [11]:
! streamlit run /home/ec2-user/SageMaker/app.py --server.baseUrlPath="proxy/absolute/8501"




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501/proxy/absolute/8501
  Network URL: http://172.16.87.38:8501/proxy/absolute/8501
  External URL: http://52.42.79.222:8501/proxy/absolute/8501

^C
  Stopping...


### Explanation:
#### Purpose of the Echo Bot:

This simple application echoes back whatever the user types. It's a basic example to show how messages can be displayed and stored in the session state.

![](../../Bedrock-Examples/Langchain/img/echo.png)

#### Before proceeding
Be sure to stop the above cell. You can do this by highlighting it and pressing the stop button on the navigation pane at the top of the notebook

## Step 2: Integrating Bedrock and DynamoDB with Streamlit

Next, we'll replace the echo bot logic with our AI-powered chatbot that uses Bedrock for generating responses and DynamoDB for managing chat history.

In [12]:
%%writefile app.py
import boto3
import logging
import streamlit as st
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_aws import ChatBedrockConverse
client = boto3.client('sts')
logging.basicConfig(level=logging.CRITICAL)

def get_iam_user_id():    
    # Get the caller identity
    identity = client.get_caller_identity()
    
    # Extract and return the User ID
    user_id = identity['UserId']
    return user_id

model = ChatBedrockConverse(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    max_tokens=2048,
    temperature=0.0,
    top_p=1,
    stop_sequences=["\n\nHuman"],
    verbose=True
)

# Initialize the DynamoDB chat message history
table_name = "SessionTable"
session_id = get_iam_user_id()  # You can make this dynamic based on the user session
history = DynamoDBChatMessageHistory(table_name=table_name, session_id=session_id)

# Create the chat prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

output_parser = StrOutputParser()

# Combine the prompt with the Bedrock LLM
chain = prompt_template | model | output_parser

# Integrate with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: DynamoDBChatMessageHistory(
        table_name=table_name, session_id=session_id
    ),
    input_messages_key="question",
    history_messages_key="history",
)

st.title("LangChain DynamoDB Bot")

# Load messages from DynamoDB and populate chat history
if "messages" not in st.session_state:
    st.session_state.messages = []
    
    # Load the stored messages from DynamoDB
    stored_messages = history.messages  # Retrieve all stored messages
    
    # Populate the session state with the retrieved messages
    for msg in stored_messages:
        role = "user" if msg.__class__.__name__ == "HumanMessage" else "assistant"
        st.session_state.messages.append({"role": role, "content": msg.content})

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("What is up?"):
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    # Generate assistant response using Bedrock LLM and LangChain
    config = {"configurable": {"session_id": session_id}}
    response = chain_with_history.invoke({"question": prompt}, config=config)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})


Overwriting app.py


### ## Running the Application

Finally, you can run your Streamlit application with the following command. This will launch the Streamlit application and allow you to interact with the chatbot through a web interface.

Take your notebook URL from the current web page and format it like the following in another tab in your internet browser:

`https://<unique_notebook_id>.notebook.us-west-2.sagemaker.aws/proxy/absolute/8501`

In [ ]:
! streamlit run /home/ec2-user/SageMaker/app.py --server.baseUrlPath="proxy/absolute/8501"




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501/proxy/absolute/8501
  Network URL: http://172.16.87.38:8501/proxy/absolute/8501
  External URL: http://52.42.79.222:8501/proxy/absolute/8501



### Explanation:

The Streamlit app now interacts with the Bedrock model and DynamoDB to provide responses that maintain context over time. Each session is uniquely identified by the IAM User ID, ensuring that different users or roles have separate conversation histories.

#### Streamlit UI:
The interface is designed to display past conversations and generate new responses based on user input. This makes it easy to interact with the chatbot and see the context-aware responses.

![](../../Bedrock-Examples/Langchain/img/chatbot.png)